In [1]:
import torch

Empty makes an uninitialised array. This could have any numbers.

In [2]:
x = torch.empty(5, 3)

In [5]:
print(x)

tensor([[7.4531e+20, 4.5687e-41, 7.4531e+20],
        [4.5687e-41, 1.8174e+31, 1.9519e-19],
        [4.4377e+27, 1.9366e+31, 2.0283e-19],
        [7.1428e+31, 3.4447e-12, 1.3563e-19],
        [2.9538e+21, 7.3986e+20, 1.7312e+25]])


In [6]:
x = torch.zeros(4,3)

In [7]:
x

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [8]:
x = torch.rand(5,3)

In [9]:
x

tensor([[0.1327, 0.3382, 0.9595],
        [0.4144, 0.0566, 0.6663],
        [0.7543, 0.2198, 0.7454],
        [0.0858, 0.8118, 0.8330],
        [0.6663, 0.5409, 0.1857]])

In [10]:
x = torch.zeros(4,3, dtype=torch.long)

In [11]:
x

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

**To create a tensor from a given list**

In [12]:
x = torch.tensor([5.5, 3])

In [13]:
x

tensor([5.5000, 3.0000])

**To create a tensor from an existing tensor**

In [14]:
x = torch.randn_like(x, dtype=torch.float)

In [15]:
x

tensor([2.8764, 2.0748])

In [16]:
x.size()

torch.Size([2])

In [17]:
x = torch.randn(4,3)

In [24]:
x

tensor([[ 0.4227,  0.8795,  0.6566],
        [-1.5129, -2.0881, -0.1993],
        [ 0.3387, -2.0994, -1.8862],
        [ 0.0404,  0.5606,  0.0959]])

**Get subset of the matrix**

In [26]:
y = x.view(12)

In [27]:
x

tensor([[ 0.4227,  0.8795,  0.6566],
        [-1.5129, -2.0881, -0.1993],
        [ 0.3387, -2.0994, -1.8862],
        [ 0.0404,  0.5606,  0.0959]])

In [28]:
y

tensor([ 0.4227,  0.8795,  0.6566, -1.5129, -2.0881, -0.1993,  0.3387, -2.0994,
        -1.8862,  0.0404,  0.5606,  0.0959])

In [30]:
z = x.view(-1, 6)

In [31]:
z

tensor([[ 0.4227,  0.8795,  0.6566, -1.5129, -2.0881, -0.1993],
        [ 0.3387, -2.0994, -1.8862,  0.0404,  0.5606,  0.0959]])

### Autograd

In [32]:
x = torch.ones(2,2, requires_grad = True)

In [33]:
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [34]:
y = x + 2

In [35]:
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [36]:
y.grad_fn

In [37]:
x.grad_fn

y was created as a result of an operation, so it has a grad_fn. x was created by user, so it doesn't.

In [38]:
z = y * y * 3

In [39]:
out = z.mean()

In [41]:
z

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)

In [42]:
out

tensor(27., grad_fn=<MeanBackward1>)

In [44]:
out.backward()

In [55]:
x.grad

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])

In [57]:
y.grad

In [58]:
z.grad

So, we called `.backward()` to do backward prop and calculate gradients for each thing. And we can check those gradients value by doing `.grad`

In [47]:
a = torch.randn(2,2)

In [48]:
a = ((a*3) / (a-1))

In [49]:
a

tensor([[ 0.4947,  1.6702],
        [ 1.2396, 91.2925]])

In [50]:
a.requires_grad

False

In [51]:
a.requires_grad_(True)

tensor([[ 0.4947,  1.6702],
        [ 1.2396, 91.2925]], requires_grad=True)

In [52]:
b = (a*a).sum()

In [53]:
b.grad_fn

In [54]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


### torch.nn - NN module

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

**Define the NN**

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        # 3x3 convolutional kernel ; input image channel 1 ; output image channel 6
        self.conv1 = nn.Conv2d(1,6,3)
        # 3x3 convolutional kernel ; input image channel 6 ; output image channel 16
        self.conv2 = nn.Conv2d(6, 16, 3)
        # linear operations
        self.fc1 = nn.Linear(16*6*6, 120)          # 6 * 6 is derived from checking image dimension.
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2,2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # if size if square you can only specify a single number for max pooling
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]     # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features        

**Printing it**

In [5]:
net = Net()

In [6]:
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

You just have to define the forward function, and the backward function (where gradients are computed) is automatically defined for you using autograd. You can use any of the Tensor operations in the forward function.

In [10]:
net.parameters()

<generator object Module.parameters at 0x7f1374dfda20>

In [12]:
learnable_params = list(net.parameters())

In [13]:
len(learnable_params)

10

In [14]:
learnable_params[0].size()

torch.Size([6, 1, 3, 3])

In [15]:
learnable_params[1].size()

torch.Size([6])

In [21]:
!nvidia-smi

Mon Aug 12 10:45:06 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.74       Driver Version: 418.74       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:0A:00.0 Off |                  N/A |
| 48%   50C    P2   123W / 250W |  10774MiB / 11178MiB |     25%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:41:00.0 Off |                  N/A |
| 48%   48C    P8    19W / 250W |    147MiB / 11176MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        